In [1]:
import pandas as pd

final_df = pd.read_csv("pubmed_abstract_data.csv",index_col=False)

In [16]:
final_df.to_csv("pubmed_abstract_data.csv",index=False)

In [2]:
print(final_df['abstract'].values[100])

UNLABELLED: What's known on the subject? and What does the study add? Previous reports, with small numbers of patients, have described the problem of incomplete testosterone suppression (>1.1 or 1.7 nmol/L) with LHRH agonists. Various predisposing factors have been suggested: different drug agents and patient factors such as age, pretreatment testosterone levels and weight. Such incomplete testosterone suppression has been shown in one small report to be associated with increased PSA failure rates and in another report in those with metastases, with worse survival. This study used testosterone assays that are more accurate at low levels than those used in most previous reports in a large dataset of 2196 men, and confirmed incomplete testosterone suppression (breakthrough) rates >1.7 nmol/L of 3.4% and >1.1 nmol/L of 6.6%. We showed that younger age was strongly associated with the risk of breakthrough, with a minor effect of increasing body mass index. Repeated breakthroughs were more 

In [3]:
final_df['abstract'].values[100].split("\n")[0].split(": ")[1]

"What's known on the subject? and What does the study add? Previous reports, with small numbers of patients, have described the problem of incomplete testosterone suppression (>1.1 or 1.7 nmol/L) with LHRH agonists. Various predisposing factors have been suggested"

In [8]:
true_df = final_df[final_df['label']==True]

,url,pid,abstract,label
27000,https://pubmed.ncbi.nlm.nih.gov/21645374/,21645374,BACKGROUND: Programmed cell death (PCD) is the...,True
27001,https://pubmed.ncbi.nlm.nih.gov/16418930/,16418930,BACKGROUND: Assessment of visual acuity depend...,True
27002,https://pubmed.ncbi.nlm.nih.gov/9488747/,9488747,BACKGROUND: Apparent life-threatening events i...,True
27003,https://pubmed.ncbi.nlm.nih.gov/17208539/,17208539,PURPOSE: The transanal endorectal pull-through...,True
27004,https://pubmed.ncbi.nlm.nih.gov/10808977/,10808977,BACKGROUND: Telephone counseling and tailored ...,True
...,...,...,...,...
27995,https://pubmed.ncbi.nlm.nih.gov/8921484/,8921484,"BACKGROUND: After 34 weeks gestation, summary ...",True
27996,https://pubmed.ncbi.nlm.nih.gov/16564683/,16564683,OBJECTIVE: To evaluate the accuracy of ultraso...,True
27997,https://pubmed.ncbi.nlm.nih.gov/23147106/,23147106,BACKGROUND: We analyzed the pharmacokinetic-ph...,True
27998,https://pubmed.ncbi.nlm.nih.gov/21550158/,21550158,PURPOSE: This investigation assesses the effec...,True


In [11]:
final_df[27000:32000]

,url,pid,abstract,label
27000,https://pubmed.ncbi.nlm.nih.gov/21645374/,21645374,BACKGROUND: Programmed cell death (PCD) is the...,True
27001,https://pubmed.ncbi.nlm.nih.gov/16418930/,16418930,BACKGROUND: Assessment of visual acuity depend...,True
27002,https://pubmed.ncbi.nlm.nih.gov/9488747/,9488747,BACKGROUND: Apparent life-threatening events i...,True
27003,https://pubmed.ncbi.nlm.nih.gov/17208539/,17208539,PURPOSE: The transanal endorectal pull-through...,True
27004,https://pubmed.ncbi.nlm.nih.gov/10808977/,10808977,BACKGROUND: Telephone counseling and tailored ...,True
...,...,...,...,...
31995,16148658,https://pubmed.ncbi.nlm.nih.gov/16148658/,PURPOSE: We review the outcome of stopping pro...,False
31996,16148659,https://pubmed.ncbi.nlm.nih.gov/16148659/,PURPOSE: The presence of congenital para-urete...,False
31997,16148665,https://pubmed.ncbi.nlm.nih.gov/16148665/,PURPOSE: Traditional treatment of dysfunctiona...,False
31998,16148667,https://pubmed.ncbi.nlm.nih.gov/16148667/,PURPOSE: It has been suggested that in childre...,False


In [12]:
pubmed_texts = []
faulty = []
for idx,i in enumerate(final_df[27000:32000].iterrows()):
    abstract_list = i[1]['abstract'].split("\n")
    try:
        abstract_list = [al.split(": ")[1] for al in abstract_list if len(al.split(": "))>0]
        pubmed_texts.append(" ".join(abstract_list))
    except:
        print(idx)
        faulty.append(i[1])
        print(abstract_list)

In [13]:
len(pubmed_texts)

5000

In [14]:
# pubmed_texts.append("The association between differentiated thyroid carcinoma (DTC) and chronic lymphocytic thyroiditis (CLT) has been reported in literature.  To evaluate the incidence of this association and to determine whether the CLT may influence on the early initial staging of DTC when associated with other variable risks.  Historical (retrospective) cohort.  Fifty two patients with DTC were evaluated from 1999 to 2009. They were divided into two groups. The first group had 35 patients with DTC without DLT; the second had 17 patients with CLT. Total thyroidectomy was the treatment chosen for all patients. Similarities shared in both groups such as age, gender, histological tumor type, tumor diameter, regional only or with distant metastases, extrathyroidal invasion, multifocality and presence of tumor capsule were considered. T-Student tests and Chi-square tests were applied to analyze the data.  The incidence of DTC without CLT was higher that of DTC+CLT (p=0.0126). We noticed no statistic differences between the common variables analyzed.  CLT occurred in 33% of the patients with DCT. All cases of DTC were in the early stages.")
# pubmed_texts.append("Abnormalities in craniofacial morphology are associated with Eustachian tube dysfunction and otitis media with effusion (OME).  to evaluate the relationship between facial pattern and craniofacial growth direction, and OME in children with enlarged tonsils and adenoids (ETA).  Clinical prospective survey in 79 children (41 male and 38 female), ranging from 4 to 10 years of age, with tonsil and adenoid enlargement (Brodsky's grades III and IV). Forty children presented with OME (study group) and 39 did not (control group). Cephalometric analysis was used to determine the facial pattern.  There was no correlation observed between facial pattern and OME (c 2 = 0.25  = 0.88). Facial Axis was larger in the OME group (F(1.75) = 3.68  = 0.05) and the Lower Anterior Facial height was smaller (F(1. 75) = 3.99  = 0.05) in children with otitis media with effusion.  There was no correlation between OME and facial pattern in children with ETA although a more horizontal facial growth direction, and a smaller lower anterior facial height was observed consistently among subjects in this group. This suggests that abnormal positioning of the eustachian tube influences the development of OME in children with ETA.")

In [18]:
with open("pubmed_subset.txt",'w') as f:
    for txt in pubmed_texts:
        f.write(txt)
        f.write("\n<END>\n")

In [19]:
with open("pubmed_subset.txt",'r') as f:
    all_text = f.read()

In [21]:
len(all_text.split("<END>")[:-1])

5000

In [22]:
all_text.split("<END>")[-2]

'\nWith the constant advances in technology and expertise of prenatal diagnosis, necessity of early counseling in cases of bilateral obstructive uropathy has become of paramount importance. To better evaluate fetal renal function new biochemical (serum and urine) fetal markers have been introduced in the literature. However, they are only available at select centers and always requiring an invasive approach. Furthermore, paucity of normal controls sometimes makes interpretation of results difficult. Owing to this growing interest towards biochemical evaluation of fetal renal function, assessment of amniotic fluid, which is mostly expression of fetal urination, has progressively fallen into disrepute, and studies comparing the amount of amniotic fluid with renal function are scant. In a 3-year period 28 cases of bilateral obstructive uropathy were seen prenatally at the Artemisia Medical Center. All cases were initially reviewed at 17 to 20 weeks of gestation when a distended fetal blad